In [1]:
import os
os.chdir(r"C:\Users\lizzy\스파르타_파이썬\tmdbapi실습")  # 원하는 경로로 변경
print(os.getcwd())  # 잘 바뀌었는지 확인

C:\Users\lizzy\스파르타_파이썬\tmdbapi실습


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # 현재 디렉터리의 .env 파일 읽어서 환경변수로 등록

API_KEY = os.getenv("API_KEY")  # .env에서 API 키 읽어오기

if API_KEY is None:
    raise ValueError("API_KEY가 설정되지 않았습니다. .env 파일에 API_KEY를 확인하세요.")

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

HEADERS = {"accept": "application/json"}

# 세션 생성
def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=50,
        pool_maxsize=50
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

session = create_session()


# ----------------------------------------------------------
# 1) TV 시리즈 목록 수집
# ----------------------------------------------------------
def fetch_single_page(page, start_date, end_date):
    """단일 페이지를 가져오는 함수"""
    base_url = "https://api.themoviedb.org/3/discover/tv"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": "popularity.desc",
        "first_air_date.gte": start_date,
        "first_air_date.lte": end_date,
        "page": page,
    }
    
    try:
        response = session.get(base_url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        return data.get("results", []), data.get("total_pages", 1)
    except Exception as e:
        print(f"  ⚠️ 페이지 {page} 오류: {e}")
        return [], 1


def fetch_tv_series_between_dates(start_date, end_date):
    """특정 기간 내 TV 시리즈 목록 수집 (최대 500페이지)"""
    results, total_pages = fetch_single_page(1, start_date, end_date)
    all_series = results
    
    max_pages = min(total_pages, 500)
    print(f"  📊 총 {total_pages}페이지 → 수집: {max_pages}페이지")
    
    if max_pages == 1:
        return all_series
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = {
            executor.submit(fetch_single_page, page, start_date, end_date): page 
            for page in range(2, max_pages + 1)
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="  TV 목록 수집", leave=False):
            page_results, _ = future.result()
            all_series.extend(page_results)
    
    return all_series


# ----------------------------------------------------------
# 2) TV 시리즈 기본 정보 수집 (시즌 수 확인용)
# ----------------------------------------------------------
def fetch_tv_basic_info(series_id):
    """TV 시리즈 기본 정보 수집 (시즌 수 확인)"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        
        return {
            "series_id": data.get("id"),
            "series_name": data.get("name"),
            "number_of_seasons": data.get("number_of_seasons", 0),
            "first_air_date": data.get("first_air_date"),
            "status": data.get("status")
        }
    except Exception as e:
        return None


# ----------------------------------------------------------
# 3) TV 시즌 상세 정보 수집
# ----------------------------------------------------------
def fetch_season_details(series_id, season_number, series_name=""):
    """TV 시즌 상세 정보 수집"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}/season/{season_number}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        
        if response.status_code != 200:
            return None
        
        data = response.json()
        
        # 네트워크 정보 추출
        networks = data.get("networks", [])
        network_names = ", ".join([n.get("name", "") for n in networks])
        network_ids = ", ".join([str(n.get("id", "")) for n in networks])
        
        # 에피소드 정보 요약
        episodes = data.get("episodes", [])
        total_episodes = len(episodes)
        avg_episode_runtime = sum([ep.get("runtime", 0) for ep in episodes if ep.get("runtime")]) / max(total_episodes, 1)
        
        record = {
            "_id": data.get("_id"),
            "season_id": data.get("id"),
            "series_id": series_id,
            "series_name": series_name,
            "season_number": data.get("season_number"),
            "name": data.get("name"),
            "air_date": data.get("air_date"),
            "overview": data.get("overview"),
            "vote_average": data.get("vote_average"),
            "vote_count": data.get("vote_count"),
            "network_names": network_names,
            "network_ids": network_ids,
            "total_episodes": total_episodes,
            "avg_episode_runtime": round(avg_episode_runtime, 1) if avg_episode_runtime else None
        }
        
        return record
        
    except Exception as e:
        return None


# ----------------------------------------------------------
# 4) MAIN 실행부
# ----------------------------------------------------------
if __name__ == "__main__":
    print("=" * 90)
    print("🚀 TV SEASONS 상세 정보 수집 (2024.05 ~ 2025.05)")
    print("=" * 90)
    
    start_date = "2005-01-01"
    end_date = "2015-12-31"
    
    total_start = time.time()
    
    # STEP 1: TV 시리즈 목록 수집
    print(f"\n📅 [STEP 1] TV 시리즈 목록 수집: {start_date} ~ {end_date}")
    print("="*90)
    
    tv_series = fetch_tv_series_between_dates(start_date, end_date)
    
    if not tv_series:
        print("\n⚠️ 수집된 TV 시리즈 없음. 종료합니다.")
        exit()
    
    # 중복 제거
    unique_series = {s["id"]: s for s in tv_series}
    tv_list = list(unique_series.values())
    
    print(f"\n✅ TV 시리즈 수집 완료: {len(tv_list):,}개")
    
    # STEP 2: TV 시리즈 기본 정보 수집 (시즌 수 확인)
    print(f"\n📅 [STEP 2] TV 시리즈 기본 정보 수집 (시즌 수 확인)")
    print("="*90)
    
    series_info_list = []
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = {executor.submit(fetch_tv_basic_info, s["id"]): s for s in tv_list}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시리즈 정보 수집"):
            info = future.result()
            if info:
                series_info_list.append(info)
    
    print(f"\n✅ 시리즈 정보 수집 완료: {len(series_info_list):,}개")
    
    # STEP 3: 모든 시즌에 대한 작업 목록 생성
    print(f"\n📅 [STEP 3] 시즌 정보 수집 작업 목록 생성")
    print("="*90)
    
    season_tasks = []
    total_seasons = 0
    
    for series_info in series_info_list:
        series_id = series_info["series_id"]
        series_name = series_info["series_name"]
        num_seasons = series_info["number_of_seasons"]
        
        for season_num in range(1, num_seasons + 1):
            season_tasks.append({
                "series_id": series_id,
                "series_name": series_name,
                "season_number": season_num
            })
            total_seasons += 1
    
    print(f"✅ 총 {len(series_info_list):,}개 시리즈의 {total_seasons:,}개 시즌 작업 생성")
    
    # STEP 4: 시즌 상세 정보 수집 (멀티스레드)
    print(f"\n📅 [STEP 4] 시즌 상세 정보 수집 (멀티스레드)")
    print("="*90)
    
    season_results = []
    
    with ThreadPoolExecutor(max_workers=40) as executor:
        futures = {
            executor.submit(
                fetch_season_details, 
                task["series_id"], 
                task["season_number"],
                task["series_name"]
            ): task 
            for task in season_tasks
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시즌 상세 수집"):
            detail = future.result()
            if detail:
                season_results.append(detail)
    
    print(f"\n✅ 시즌 정보 수집 완료: {len(season_results):,}개")
    
    # STEP 5: CSV 저장
    if season_results:
        df = pd.DataFrame(season_results)
        df = df.sort_values(["series_name", "season_number"], ascending=[True, True])
        
        filename = "tv_seasons_2005_01_to_2015_12.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        
        elapsed = time.time() - total_start
        
        print("\n" + "="*90)
        print("========================== DONE ==========================")
        print(f"📌 수집된 TV 시리즈: {len(series_info_list):,}개")
        print(f"📌 수집된 시즌: {len(season_results):,}개")
        print(f"⏱  소요시간: {elapsed/60:.2f}분 ({elapsed:.2f}초)")
        print(f"💾 저장 파일: {filename}")
        print(f"\n📊 수집된 컬럼:")
        print(f"   ✅ _id: TMDB 내부 ID")
        print(f"   ✅ season_id: 시즌 고유 ID")
        print(f"   ✅ series_id: 시리즈 ID")
        print(f"   ✅ series_name: 시리즈 이름")
        print(f"   ✅ season_number: 시즌 번호")
        print(f"   ✅ name: 시즌 이름")
        print(f"   ✅ air_date: 첫 방영일")
        print(f"   ✅ overview: 시즌 개요")
        print(f"   ✅ vote_average: 평균 평점")
        print(f"   ✅ vote_count: 투표 수")
        print(f"   ✅ network_names: 방송사 이름")
        print(f"   ✅ network_ids: 방송사 ID")
        print(f"   ✅ total_episodes: 총 에피소드 수")
        print(f"   ✅ avg_episode_runtime: 평균 에피소드 런타임")
        print("="*90)
    else:
        print("\n⚠️ 저장할 데이터가 없습니다.")

🚀 TV SEASONS 상세 정보 수집 (2024.05 ~ 2025.05)

📅 [STEP 1] TV 시리즈 목록 수집: 2005-01-01 ~ 2015-12-31
  📊 총 2194페이지 → 수집: 500페이지



✅ TV 시리즈 수집 완료: 10,000개

📅 [STEP 2] TV 시리즈 기본 정보 수집 (시즌 수 확인)


시리즈 정보 수집:  26%|███████████████▏                                          | 2625/10000 [00:21<01:00, 122.56it/s]


In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

HEADERS = {"accept": "application/json"}

# 세션 생성
def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=50,
        pool_maxsize=50
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

session = create_session()


# ----------------------------------------------------------
# 헬퍼 함수 - 기간 분할
# ----------------------------------------------------------
def generate_date_periods(start_date, end_date, months=3):
    """
    시작일과 종료일 사이를 N개월 단위로 분할
    """
    periods = []
    current = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    while current <= end_dt:
        # 다음 기간 계산 (N개월 후의 1일)
        year = current.year
        month = current.month + months
        
        # 월이 12를 넘으면 연도 증가
        while month > 12:
            month -= 12
            year += 1
        
        next_date = datetime(year, month, 1)
        # 기간의 마지막 날은 다음 기간 시작 전날
        period_end = min(next_date - timedelta(days=1), end_dt)
        
        periods.append((
            current.strftime("%Y-%m-%d"),
            period_end.strftime("%Y-%m-%d")
        ))
        
        current = next_date
    
    return periods


# ----------------------------------------------------------
# 1) TV 시리즈 목록 수집
# ----------------------------------------------------------
def fetch_single_page(page, start_date, end_date):
    """단일 페이지를 가져오는 함수"""
    base_url = "https://api.themoviedb.org/3/discover/tv"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": "popularity.desc",
        "first_air_date.gte": start_date,
        "first_air_date.lte": end_date,
        "page": page,
    }
    
    try:
        response = session.get(base_url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)
    except Exception as e:
        print(f"  ⚠️ 페이지 {page} 오류: {e}")
        return [], 1, 0


def fetch_tv_series_between_dates(start_date, end_date):
    """특정 기간 내 TV 시리즈 목록 수집 (최대 500페이지)"""
    results, total_pages, total_results = fetch_single_page(1, start_date, end_date)
    all_series = results
    
    max_pages = min(total_pages, 500)
    print(f"  📊 총 {total_results:,}개 ({total_pages}페이지) → 수집: {max_pages}페이지")
    
    if total_pages > 500:
        print(f"  ⚠️  500페이지 제한으로 인해 {(total_pages - 500) * 20:,}개는 수집 불가")
    
    if max_pages == 1:
        return all_series
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = {
            executor.submit(fetch_single_page, page, start_date, end_date): page 
            for page in range(2, max_pages + 1)
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="  TV 목록 수집", leave=False):
            page_results, _, _ = future.result()
            all_series.extend(page_results)
    
    return all_series


# ----------------------------------------------------------
# 2) TV 시리즈 기본 정보 수집 (시즌 수 확인용)
# ----------------------------------------------------------
def fetch_tv_basic_info(series_id):
    """TV 시리즈 기본 정보 수집 (시즌 수 확인)"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        
        return {
            "series_id": data.get("id"),
            "series_name": data.get("name"),
            "number_of_seasons": data.get("number_of_seasons", 0),
            "first_air_date": data.get("first_air_date"),
            "status": data.get("status")
        }
    except Exception as e:
        return None


# ----------------------------------------------------------
# 3) TV 시즌 상세 정보 수집
# ----------------------------------------------------------
def fetch_season_details(series_id, season_number, series_name=""):
    """TV 시즌 상세 정보 수집"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}/season/{season_number}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        
        if response.status_code != 200:
            return None
        
        data = response.json()
        
        # 네트워크 정보 추출
        networks = data.get("networks", [])
        network_names = ", ".join([n.get("name", "") for n in networks])
        network_ids = ", ".join([str(n.get("id", "")) for n in networks])
        
        # 에피소드 정보 요약
        episodes = data.get("episodes", [])
        total_episodes = len(episodes)
        avg_episode_runtime = sum([ep.get("runtime", 0) for ep in episodes if ep.get("runtime")]) / max(total_episodes, 1)
        
        record = {
            "_id": data.get("_id"),
            "season_id": data.get("id"),
            "series_id": series_id,
            "series_name": series_name,
            "season_number": data.get("season_number"),
            "name": data.get("name"),
            "air_date": data.get("air_date"),
            "overview": data.get("overview"),
            "poster_path": data.get("poster_path"),
            "vote_average": data.get("vote_average"),
            "vote_count": data.get("vote_count"),
            "network_names": network_names,
            "network_ids": network_ids,
            "total_episodes": total_episodes,
            "avg_episode_runtime": round(avg_episode_runtime, 1) if avg_episode_runtime else None
        }
        
        return record
        
    except Exception as e:
        return None


# ----------------------------------------------------------
# 4) MAIN 실행부
# ----------------------------------------------------------
if __name__ == "__main__":
    print("=" * 90)
    print("🚀 TV SEASONS 상세 정보 수집 (2024.05 ~ 2025.05)")
    print("=" * 90)
    
    start_date = "2024-05-01"
    end_date = "2025-05-31"
    
    total_start = time.time()
    
    # STEP 1: TV 시리즈 목록 수집 (3개월씩 분할)
    print(f"\n📅 [STEP 1] TV 시리즈 목록 수집: {start_date} ~ {end_date}")
    print("="*90)
    
    # 기간을 3개월 단위로 분할
    periods = generate_date_periods(start_date, end_date, months=3)
    
    print(f"\n📊 전체 기간을 {len(periods)}개 구간으로 분할 (각 3개월씩)")
    print("="*90)
    
    all_tv_series = []
    
    # 각 기간별로 데이터 수집
    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 📌 기간: {period_start} ~ {period_end}")
        
        tv_series = fetch_tv_series_between_dates(period_start, period_end)
        all_tv_series.extend(tv_series)
        
        print(f"  ✅ 이 기간 수집: {len(tv_series):,}개 (누적: {len(all_tv_series):,}개)")
    
    if not all_tv_series:
        print("\n⚠️ 수집된 TV 시리즈 없음. 종료합니다.")
        exit()
    
    # 중복 제거
    print(f"\n{'='*90}")
    print(f"🧹 중복 제거 중...")
    unique_series = {s["id"]: s for s in all_tv_series}
    tv_list = list(unique_series.values())
    print(f"✨ 중복 제거 전: {len(all_tv_series):,}개 → 중복 제거 후: {len(tv_list):,}개")
    print(f"   (제거된 중복: {len(all_tv_series) - len(tv_list):,}개)")
    
    print(f"\n✅ TV 시리즈 수집 완료: {len(tv_list):,}개")
    
    # STEP 2: TV 시리즈 기본 정보 수집 (시즌 수 확인)
    print(f"\n📅 [STEP 2] TV 시리즈 기본 정보 수집 (시즌 수 확인)")
    print("="*90)
    
    series_info_list = []
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = {executor.submit(fetch_tv_basic_info, s["id"]): s for s in tv_list}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시리즈 정보 수집"):
            info = future.result()
            if info:
                series_info_list.append(info)
    
    print(f"\n✅ 시리즈 정보 수집 완료: {len(series_info_list):,}개")
    
    # STEP 3: 모든 시즌에 대한 작업 목록 생성
    print(f"\n📅 [STEP 3] 시즌 정보 수집 작업 목록 생성")
    print("="*90)
    
    season_tasks = []
    total_seasons = 0
    
    for series_info in series_info_list:
        series_id = series_info["series_id"]
        series_name = series_info["series_name"]
        num_seasons = series_info["number_of_seasons"]
        
        for season_num in range(1, num_seasons + 1):
            season_tasks.append({
                "series_id": series_id,
                "series_name": series_name,
                "season_number": season_num
            })
            total_seasons += 1
    
    print(f"✅ 총 {len(series_info_list):,}개 시리즈의 {total_seasons:,}개 시즌 작업 생성")
    
    # STEP 4: 시즌 상세 정보 수집 (멀티스레드)
    print(f"\n📅 [STEP 4] 시즌 상세 정보 수집 (멀티스레드)")
    print("="*90)
    
    season_results = []
    
    with ThreadPoolExecutor(max_workers=40) as executor:
        futures = {
            executor.submit(
                fetch_season_details, 
                task["series_id"], 
                task["season_number"],
                task["series_name"]
            ): task 
            for task in season_tasks
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시즌 상세 수집"):
            detail = future.result()
            if detail:
                season_results.append(detail)
    
    print(f"\n✅ 시즌 정보 수집 완료: {len(season_results):,}개")
    
    # STEP 5: CSV 저장
    if season_results:
        df = pd.DataFrame(season_results)
        df = df.sort_values(["series_name", "season_number"], ascending=[True, True])
        
        filename = "tv_seasons_2024_05_to_2025_05.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        
        elapsed = time.time() - total_start
        
        print("\n" + "="*90)
        print("========================== DONE ==========================")
        print(f"📌 수집된 TV 시리즈: {len(series_info_list):,}개")
        print(f"📌 수집된 시즌: {len(season_results):,}개")
        print(f"⏱  소요시간: {elapsed/60:.2f}분 ({elapsed:.2f}초)")
        print(f"💾 저장 파일: {filename}")
        print(f"\n📊 수집된 컬럼:")
        print(f"   ✅ _id: TMDB 내부 ID")
        print(f"   ✅ season_id: 시즌 고유 ID")
        print(f"   ✅ series_id: 시리즈 ID")
        print(f"   ✅ series_name: 시리즈 이름")
        print(f"   ✅ season_number: 시즌 번호")
        print(f"   ✅ name: 시즌 이름")
        print(f"   ✅ air_date: 첫 방영일")
        print(f"   ✅ overview: 시즌 개요")
        print(f"   ✅ poster_path: 포스터 경로")
        print(f"   ✅ vote_average: 평균 평점")
        print(f"   ✅ vote_count: 투표 수")
        print(f"   ✅ network_names: 방송사 이름")
        print(f"   ✅ network_ids: 방송사 ID")
        print(f"   ✅ total_episodes: 총 에피소드 수")
        print(f"   ✅ avg_episode_runtime: 평균 에피소드 런타임")
        print("="*90)
    else:
        print("\n⚠️ 저장할 데이터가 없습니다.")

🚀 TV SEASONS 상세 정보 수집 (2024.05 ~ 2025.05)

📅 [STEP 1] TV 시리즈 목록 수집: 2024-05-01 ~ 2025-05-31

📊 전체 기간을 5개 구간으로 분할 (각 3개월씩)

[1/5] 📌 기간: 2024-05-01 ~ 2024-07-31
  📊 총 3,203개 (161페이지) → 수집: 161페이지


  ✅ 이 기간 수집: 3,203개 (누적: 3,203개)

[2/5] 📌 기간: 2024-08-01 ~ 2024-10-31
  📊 총 3,724개 (187페이지) → 수집: 187페이지


  ✅ 이 기간 수집: 3,724개 (누적: 6,927개)

[3/5] 📌 기간: 2024-11-01 ~ 2025-01-31
  📊 총 3,309개 (166페이지) → 수집: 166페이지


  ✅ 이 기간 수집: 3,309개 (누적: 10,236개)

[4/5] 📌 기간: 2025-02-01 ~ 2025-04-30
  📊 총 2,883개 (145페이지) → 수집: 145페이지


  ✅ 이 기간 수집: 2,883개 (누적: 13,119개)

[5/5] 📌 기간: 2025-05-01 ~ 2025-05-31
  📊 총 842개 (43페이지) → 수집: 43페이지


  ✅ 이 기간 수집: 842개 (누적: 13,961개)

🧹 중복 제거 중...
✨ 중복 제거 전: 13,961개 → 중복 제거 후: 13,961개
   (제거된 중복: 0개)

✅ TV 시리즈 수집 완료: 13,961개

📅 [STEP 2] TV 시리즈 기본 정보 수집 (시즌 수 확인)


시리즈 정보 수집:  72%|██████████████████████████████████████████▏                | 9995/13961 [01:56<00:46, 85.83it/s]


In [3]:
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

HEADERS = {"accept": "application/json"}

# 세션 생성
def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=50,
        pool_maxsize=50
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

session = create_session()


# ----------------------------------------------------------
# 헬퍼 함수 - 기간 분할
# ----------------------------------------------------------
def generate_date_periods(start_date, end_date, months=3):
    """
    시작일과 종료일 사이를 N개월 단위로 분할
    """
    periods = []
    current = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    while current <= end_dt:
        # 다음 기간 계산 (N개월 후의 1일)
        year = current.year
        month = current.month + months
        
        # 월이 12를 넘으면 연도 증가
        while month > 12:
            month -= 12
            year += 1
        
        next_date = datetime(year, month, 1)
        # 기간의 마지막 날은 다음 기간 시작 전날
        period_end = min(next_date - timedelta(days=1), end_dt)
        
        periods.append((
            current.strftime("%Y-%m-%d"),
            period_end.strftime("%Y-%m-%d")
        ))
        
        current = next_date
    
    return periods


# ----------------------------------------------------------
# 1) TV 시리즈 목록 수집
# ----------------------------------------------------------
def fetch_single_page(page, start_date, end_date):
    """단일 페이지를 가져오는 함수"""
    base_url = "https://api.themoviedb.org/3/discover/tv"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": "popularity.desc",
        "first_air_date.gte": start_date,
        "first_air_date.lte": end_date,
        "page": page,
    }
    
    try:
        response = session.get(base_url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)
    except Exception as e:
        print(f"  ⚠️ 페이지 {page} 오류: {e}")
        return [], 1, 0


def fetch_tv_series_between_dates(start_date, end_date):
    """특정 기간 내 TV 시리즈 목록 수집 (최대 500페이지)"""
    results, total_pages, total_results = fetch_single_page(1, start_date, end_date)
    all_series = results
    
    max_pages = min(total_pages, 500)
    print(f"  📊 총 {total_results:,}개 ({total_pages}페이지) → 수집: {max_pages}페이지")
    
    if total_pages > 500:
        print(f"  ⚠️  500페이지 제한으로 인해 {(total_pages - 500) * 20:,}개는 수집 불가")
    
    if max_pages == 1:
        return all_series
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = {
            executor.submit(fetch_single_page, page, start_date, end_date): page 
            for page in range(2, max_pages + 1)
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="  TV 목록 수집", leave=False):
            page_results, _, _ = future.result()
            all_series.extend(page_results)
    
    return all_series


# ----------------------------------------------------------
# 2) TV 시리즈 기본 정보 수집 (시즌 수 확인용)
# ----------------------------------------------------------
def fetch_tv_basic_info(series_id):
    """TV 시리즈 기본 정보 수집 (시즌 수 확인)"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        
        return {
            "series_id": data.get("id"),
            "series_name": data.get("name"),
            "number_of_seasons": data.get("number_of_seasons", 0),
            "first_air_date": data.get("first_air_date"),
            "status": data.get("status")
        }
    except Exception as e:
        return None


# ----------------------------------------------------------
# 3) TV 시즌 상세 정보 수집
# ----------------------------------------------------------
def fetch_season_details(series_id, season_number, series_name=""):
    """TV 시즌 상세 정보 수집"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}/season/{season_number}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        
        if response.status_code != 200:
            return None
        
        data = response.json()
        
        # 네트워크 정보 추출
        networks = data.get("networks", [])
        network_names = ", ".join([n.get("name", "") for n in networks])
        network_ids = ", ".join([str(n.get("id", "")) for n in networks])
        
        # 에피소드 정보 요약
        episodes = data.get("episodes", [])
        total_episodes = len(episodes)
        avg_episode_runtime = sum([ep.get("runtime", 0) for ep in episodes if ep.get("runtime")]) / max(total_episodes, 1)
        
        record = {
            "_id": data.get("_id"),
            "season_id": data.get("id"),
            "series_id": series_id,
            "series_name": series_name,
            "season_number": data.get("season_number"),
            "name": data.get("name"),
            "air_date": data.get("air_date"),
            "overview": data.get("overview"),
            "vote_average": data.get("vote_average"),
            "vote_count": data.get("vote_count"),
            "network_names": network_names,
            "network_ids": network_ids,
            "total_episodes": total_episodes,
            "avg_episode_runtime": round(avg_episode_runtime, 1) if avg_episode_runtime else None
        }
        
        return record
        
    except Exception as e:
        return None


# ----------------------------------------------------------
# 4) MAIN 실행부
# ----------------------------------------------------------
if __name__ == "__main__":
    print("=" * 90)
    print("🚀 TV SEASONS 상세 정보 수집 (2005.01 ~ 2015.12)")
    print("=" * 90)
    
    start_date = "2005-01-01"
    end_date = "2015-12-31"
    
    total_start = time.time()
    
    # STEP 1: TV 시리즈 목록 수집 (3개월씩 분할)
    print(f"\n📅 [STEP 1] TV 시리즈 목록 수집: {start_date} ~ {end_date}")
    print("="*90)
    
    # 기간을 3개월 단위로 분할
    periods = generate_date_periods(start_date, end_date, months=3)
    
    print(f"\n📊 전체 기간을 {len(periods)}개 구간으로 분할 (각 3개월씩)")
    print("="*90)
    
    all_tv_series = []
    
    # 각 기간별로 데이터 수집
    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 📌 기간: {period_start} ~ {period_end}")
        
        tv_series = fetch_tv_series_between_dates(period_start, period_end)
        all_tv_series.extend(tv_series)
        
        print(f"  ✅ 이 기간 수집: {len(tv_series):,}개 (누적: {len(all_tv_series):,}개)")
    
    if not all_tv_series:
        print("\n⚠️ 수집된 TV 시리즈 없음. 종료합니다.")
        exit()
    
    # 중복 제거
    print(f"\n{'='*90}")
    print(f"🧹 중복 제거 중...")
    unique_series = {s["id"]: s for s in all_tv_series}
    tv_list = list(unique_series.values())
    print(f"✨ 중복 제거 전: {len(all_tv_series):,}개 → 중복 제거 후: {len(tv_list):,}개")
    print(f"   (제거된 중복: {len(all_tv_series) - len(tv_list):,}개)")
    
    print(f"\n✅ TV 시리즈 수집 완료: {len(tv_list):,}개")
    
    # STEP 2: TV 시리즈 기본 정보 수집 (시즌 수 확인)
    print(f"\n📅 [STEP 2] TV 시리즈 기본 정보 수집 (시즌 수 확인)")
    print("="*90)
    
    series_info_list = []
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = {executor.submit(fetch_tv_basic_info, s["id"]): s for s in tv_list}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시리즈 정보 수집"):
            info = future.result()
            if info:
                series_info_list.append(info)
    
    print(f"\n✅ 시리즈 정보 수집 완료: {len(series_info_list):,}개")
    
    # STEP 3: 모든 시즌에 대한 작업 목록 생성
    print(f"\n📅 [STEP 3] 시즌 정보 수집 작업 목록 생성")
    print("="*90)
    
    season_tasks = []
    total_seasons = 0
    
    for series_info in series_info_list:
        series_id = series_info["series_id"]
        series_name = series_info["series_name"]
        num_seasons = series_info["number_of_seasons"]
        
        for season_num in range(1, num_seasons + 1):
            season_tasks.append({
                "series_id": series_id,
                "series_name": series_name,
                "season_number": season_num
            })
            total_seasons += 1
    
    print(f"✅ 총 {len(series_info_list):,}개 시리즈의 {total_seasons:,}개 시즌 작업 생성")
    
    # STEP 4: 시즌 상세 정보 수집 (멀티스레드)
    print(f"\n📅 [STEP 4] 시즌 상세 정보 수집 (멀티스레드)")
    print("="*90)
    
    season_results = []
    
    with ThreadPoolExecutor(max_workers=40) as executor:
        futures = {
            executor.submit(
                fetch_season_details, 
                task["series_id"], 
                task["season_number"],
                task["series_name"]
            ): task 
            for task in season_tasks
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시즌 상세 수집"):
            detail = future.result()
            if detail:
                season_results.append(detail)
    
    print(f"\n✅ 시즌 정보 수집 완료: {len(season_results):,}개")
    
    # STEP 5: CSV 저장
    if season_results:
        df = pd.DataFrame(season_results)
        df = df.sort_values(["series_name", "season_number"], ascending=[True, True])
        
        filename = "tv_seasons_2005_2015.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        
        elapsed = time.time() - total_start
        
        print("\n" + "="*90)
        print("========================== DONE ==========================")
        print(f"📌 수집된 TV 시리즈: {len(series_info_list):,}개")
        print(f"📌 수집된 시즌: {len(season_results):,}개")
        print(f"⏱  소요시간: {elapsed/60:.2f}분 ({elapsed:.2f}초)")
        print(f"💾 저장 파일: {filename}")
        print(f"\n📊 수집된 컬럼:")
        print(f"   ✅ _id: TMDB 내부 ID")
        print(f"   ✅ season_id: 시즌 고유 ID")
        print(f"   ✅ series_id: 시리즈 ID")
        print(f"   ✅ series_name: 시리즈 이름")
        print(f"   ✅ season_number: 시즌 번호")
        print(f"   ✅ name: 시즌 이름")
        print(f"   ✅ air_date: 첫 방영일")
        print(f"   ✅ overview: 시즌 개요")
        print(f"   ✅ vote_average: 평균 평점")
        print(f"   ✅ vote_count: 투표 수")
        print(f"   ✅ network_names: 방송사 이름")
        print(f"   ✅ network_ids: 방송사 ID")
        print(f"   ✅ total_episodes: 총 에피소드 수")
        print(f"   ✅ avg_episode_runtime: 평균 에피소드 런타임")
        print("="*90)
    else:
        print("\n⚠️ 저장할 데이터가 없습니다.")

🚀 TV SEASONS 상세 정보 수집 (2005.01 ~ 2015.12)

📅 [STEP 1] TV 시리즈 목록 수집: 2005-01-01 ~ 2015-12-31

📊 전체 기간을 44개 구간으로 분할 (각 3개월씩)

[1/44] 📌 기간: 2005-01-01 ~ 2005-03-31
  📊 총 719개 (36페이지) → 수집: 36페이지


  ✅ 이 기간 수집: 719개 (누적: 719개)

[2/44] 📌 기간: 2005-04-01 ~ 2005-06-30
  📊 총 540개 (27페이지) → 수집: 27페이지


  ✅ 이 기간 수집: 540개 (누적: 1,259개)

[3/44] 📌 기간: 2005-07-01 ~ 2005-09-30
  📊 총 610개 (31페이지) → 수집: 31페이지


  ✅ 이 기간 수집: 610개 (누적: 1,869개)

[4/44] 📌 기간: 2005-10-01 ~ 2005-12-31
  📊 총 648개 (33페이지) → 수집: 33페이지


  ✅ 이 기간 수집: 648개 (누적: 2,517개)

[5/44] 📌 기간: 2006-01-01 ~ 2006-03-31
  📊 총 892개 (45페이지) → 수집: 45페이지


...

  ✅ 이 기간 수집: 1,352개 (누적: 43,869개)

🧹 중복 제거 중...
✨ 중복 제거 전: 43,869개 → 중복 제거 후: 43,869개
   (제거된 중복: 0개)

✅ TV 시리즈 수집 완료: 43,869개

📅 [STEP 2] TV 시리즈 기본 정보 수집 (시즌 수 확인)


시리즈 정보 수집: 100%|██████████████████████████████████████████████████████████| 43869/43869 [16:52<00:00, 43.34it/s]



✅ 시리즈 정보 수집 완료: 43,869개

📅 [STEP 3] 시즌 정보 수집 작업 목록 생성
✅ 총 43,869개 시리즈의 94,827개 시즌 작업 생성

📅 [STEP 4] 시즌 상세 정보 수집 (멀티스레드)


시즌 상세 수집: 100%|██████████████████████████████████████████████████████████| 94827/94827 [1:05:18<00:00, 24.20it/s]



✅ 시즌 정보 수집 완료: 93,549개

========================== DONE ==========================
📌 수집된 TV 시리즈: 43,869개
📌 수집된 시즌: 93,549개
⏱  소요시간: 83.75분 (5025.24초)
💾 저장 파일: tv_seasons_2005_2015.csv

📊 수집된 컬럼:
   ✅ _id: TMDB 내부 ID
   ✅ season_id: 시즌 고유 ID
   ✅ series_id: 시리즈 ID
   ✅ series_name: 시리즈 이름
   ✅ season_number: 시즌 번호
   ✅ name: 시즌 이름
   ✅ air_date: 첫 방영일
   ✅ overview: 시즌 개요
   ✅ vote_average: 평균 평점
   ✅ vote_count: 투표 수
   ✅ network_names: 방송사 이름
   ✅ network_ids: 방송사 ID
   ✅ total_episodes: 총 에피소드 수
   ✅ avg_episode_runtime: 평균 에피소드 런타임


In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

API_KEY = "YOUR_API_KEY_HERE"  # 여기에 API 키를 입력하세요
HEADERS = {"accept": "application/json"}

# 세션 생성
def create_session():
    """재시도 로직과 연결 풀이 있는 세션 생성"""
    session = requests.Session()
    retry = Retry(
        total=3,
        backoff_factor=0.3,
        status_forcelist=[500, 502, 503, 504]
    )
    adapter = HTTPAdapter(
        max_retries=retry,
        pool_connections=50,
        pool_maxsize=50
    )
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session

session = create_session()


# ----------------------------------------------------------
# 헬퍼 함수 - 기간 분할
# ----------------------------------------------------------
def generate_date_periods(start_date, end_date, months=3):
    """
    시작일과 종료일 사이를 N개월 단위로 분할
    """
    periods = []
    current = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    
    while current <= end_dt:
        # 다음 기간 계산 (N개월 후의 1일)
        year = current.year
        month = current.month + months
        
        # 월이 12를 넘으면 연도 증가
        while month > 12:
            month -= 12
            year += 1
        
        next_date = datetime(year, month, 1)
        # 기간의 마지막 날은 다음 기간 시작 전날
        period_end = min(next_date - timedelta(days=1), end_dt)
        
        periods.append((
            current.strftime("%Y-%m-%d"),
            period_end.strftime("%Y-%m-%d")
        ))
        
        current = next_date
    
    return periods


# ----------------------------------------------------------
# 1) TV 시리즈 목록 수집
# ----------------------------------------------------------
def fetch_single_page(page, start_date, end_date):
    """단일 페이지를 가져오는 함수"""
    base_url = "https://api.themoviedb.org/3/discover/tv"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": "popularity.desc",
        "first_air_date.gte": start_date,
        "first_air_date.lte": end_date,
        "page": page,
    }
    
    try:
        response = session.get(base_url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)
    except Exception as e:
        print(f"  ⚠️ 페이지 {page} 오류: {e}")
        return [], 1, 0


def fetch_tv_series_between_dates(start_date, end_date):
    """특정 기간 내 TV 시리즈 목록 수집 (최대 500페이지)"""
    results, total_pages, total_results = fetch_single_page(1, start_date, end_date)
    all_series = results
    
    max_pages = min(total_pages, 500)
    print(f"  📊 총 {total_results:,}개 ({total_pages}페이지) → 수집: {max_pages}페이지")
    
    if total_pages > 500:
        print(f"  ⚠️  500페이지 제한으로 인해 {(total_pages - 500) * 20:,}개는 수집 불가")
    
    if max_pages == 1:
        return all_series
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = {
            executor.submit(fetch_single_page, page, start_date, end_date): page 
            for page in range(2, max_pages + 1)
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="  TV 목록 수집", leave=False):
            page_results, _, _ = future.result()
            all_series.extend(page_results)
    
    return all_series


# ----------------------------------------------------------
# 2) TV 시리즈 기본 정보 수집 (시즌 수 확인용)
# ----------------------------------------------------------
def fetch_tv_basic_info(series_id):
    """TV 시리즈 기본 정보 수집 (시즌 수 확인)"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        data = response.json()
        
        return {
            "series_id": data.get("id"),
            "series_name": data.get("name"),
            "number_of_seasons": data.get("number_of_seasons", 0),
            "first_air_date": data.get("first_air_date"),
            "status": data.get("status")
        }
    except Exception as e:
        return None


# ----------------------------------------------------------
# 3) TV 시즌 상세 정보 수집
# ----------------------------------------------------------
def fetch_season_details(series_id, season_number, series_name=""):
    """TV 시즌 상세 정보 수집"""
    url = f"https://api.themoviedb.org/3/tv/{series_id}/season/{season_number}"
    params = {"api_key": API_KEY, "language": "en-US"}
    
    try:
        response = session.get(url, params=params, headers=HEADERS, timeout=10)
        
        if response.status_code != 200:
            return None
        
        data = response.json()
        
        # 네트워크 정보 추출
        networks = data.get("networks", [])
        network_names = ", ".join([n.get("name", "") for n in networks])
        network_ids = ", ".join([str(n.get("id", "")) for n in networks])
        
        # 에피소드 정보 요약
        episodes = data.get("episodes", [])
        total_episodes = len(episodes)
        avg_episode_runtime = sum([ep.get("runtime", 0) for ep in episodes if ep.get("runtime")]) / max(total_episodes, 1)
        
        record = {
            "_id": data.get("_id"),
            "season_id": data.get("id"),
            "series_id": series_id,
            "series_name": series_name,
            "season_number": data.get("season_number"),
            "name": data.get("name"),
            "air_date": data.get("air_date"),
            "overview": data.get("overview"),
            "vote_average": data.get("vote_average"),
            "vote_count": data.get("vote_count"),
            "network_names": network_names,
            "network_ids": network_ids,
            "total_episodes": total_episodes,
            "avg_episode_runtime": round(avg_episode_runtime, 1) if avg_episode_runtime else None
        }
        
        return record
        
    except Exception as e:
        return None


# ----------------------------------------------------------
# 4) MAIN 실행부
# ----------------------------------------------------------
if __name__ == "__main__":
    print("=" * 90)
    print("🚀 TV SEASONS 상세 정보 수집 (2005.01 ~ 2015.12)")
    print("=" * 90)
    
    start_date = "2005-01-01"
    end_date = "2015-12-31"
    
    total_start = time.time()
    
    # STEP 1: TV 시리즈 목록 수집 (3개월씩 분할)
    print(f"\n📅 [STEP 1] TV 시리즈 목록 수집: {start_date} ~ {end_date}")
    print("="*90)
    
    # 기간을 3개월 단위로 분할
    periods = generate_date_periods(start_date, end_date, months=3)
    
    print(f"\n📊 전체 기간을 {len(periods)}개 구간으로 분할 (각 3개월씩)")
    print("="*90)
    
    all_tv_series = []
    
    # 각 기간별로 데이터 수집
    for idx, (period_start, period_end) in enumerate(periods, 1):
        print(f"\n[{idx}/{len(periods)}] 📌 기간: {period_start} ~ {period_end}")
        
        tv_series = fetch_tv_series_between_dates(period_start, period_end)
        all_tv_series.extend(tv_series)
        
        print(f"  ✅ 이 기간 수집: {len(tv_series):,}개 (누적: {len(all_tv_series):,}개)")
    
    if not all_tv_series:
        print("\n⚠️ 수집된 TV 시리즈 없음. 종료합니다.")
        exit()
    
    # 중복 제거
    print(f"\n{'='*90}")
    print(f"🧹 중복 제거 중...")
    unique_series = {s["id"]: s for s in all_tv_series}
    tv_list = list(unique_series.values())
    print(f"✨ 중복 제거 전: {len(all_tv_series):,}개 → 중복 제거 후: {len(tv_list):,}개")
    print(f"   (제거된 중복: {len(all_tv_series) - len(tv_list):,}개)")
    
    print(f"\n✅ TV 시리즈 수집 완료: {len(tv_list):,}개")
    
    # STEP 2: TV 시리즈 기본 정보 수집 (시즌 수 확인)
    print(f"\n📅 [STEP 2] TV 시리즈 기본 정보 수집 (시즌 수 확인)")
    print("="*90)
    
    series_info_list = []
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = {executor.submit(fetch_tv_basic_info, s["id"]): s for s in tv_list}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시리즈 정보 수집"):
            info = future.result()
            if info:
                series_info_list.append(info)
    
    print(f"\n✅ 시리즈 정보 수집 완료: {len(series_info_list):,}개")
    
    # STEP 3: 모든 시즌에 대한 작업 목록 생성
    print(f"\n📅 [STEP 3] 시즌 정보 수집 작업 목록 생성")
    print("="*90)
    
    season_tasks = []
    total_seasons = 0
    
    for series_info in series_info_list:
        series_id = series_info["series_id"]
        series_name = series_info["series_name"]
        num_seasons = series_info["number_of_seasons"]
        
        for season_num in range(1, num_seasons + 1):
            season_tasks.append({
                "series_id": series_id,
                "series_name": series_name,
                "season_number": season_num
            })
            total_seasons += 1
    
    print(f"✅ 총 {len(series_info_list):,}개 시리즈의 {total_seasons:,}개 시즌 작업 생성")
    
    # STEP 4: 시즌 상세 정보 수집 (멀티스레드)
    print(f"\n📅 [STEP 4] 시즌 상세 정보 수집 (멀티스레드)")
    print("="*90)
    
    season_results = []
    
    with ThreadPoolExecutor(max_workers=40) as executor:
        futures = {
            executor.submit(
                fetch_season_details, 
                task["series_id"], 
                task["season_number"],
                task["series_name"]
            ): task 
            for task in season_tasks
        }
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="시즌 상세 수집"):
            detail = future.result()
            if detail:
                season_results.append(detail)
    
    print(f"\n✅ 시즌 정보 수집 완료: {len(season_results):,}개")
    
    # STEP 5: CSV 저장
    if season_results:
        df = pd.DataFrame(season_results)
        df = df.sort_values(["series_name", "season_number"], ascending=[True, True])
        
        filename = "tv_seasons_2005_2015.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        
        elapsed = time.time() - total_start
        
        print("\n" + "="*90)
        print("========================== DONE ==========================")
        print(f"📌 수집된 TV 시리즈: {len(series_info_list):,}개")
        print(f"📌 수집된 시즌: {len(season_results):,}개")
        print(f"⏱  소요시간: {elapsed/60:.2f}분 ({elapsed:.2f}초)")
        print(f"💾 저장 파일: {filename}")
        
        # 결측률 분석
        print(f"\n📊 결측률 분석:")
        print("="*90)
        total_rows = len(df)
        
        for col in df.columns:
            missing_count = df[col].isna().sum()
            missing_rate = (missing_count / total_rows) * 100
            
            # 결측률에 따라 이모지 표시
            if missing_rate == 0:
                emoji = "✅"
            elif missing_rate < 10:
                emoji = "🟢"
            elif missing_rate < 30:
                emoji = "🟡"
            elif missing_rate < 50:
                emoji = "🟠"
            else:
                emoji = "🔴"
            
            print(f"{emoji} {col:30s}: {missing_count:6,}개 ({missing_rate:6.2f}%)")
        
        print(f"\n📊 수집된 컬럼:")
        print(f"   ✅ _id: TMDB 내부 ID")
        print(f"   ✅ season_id: 시즌 고유 ID")
        print(f"   ✅ series_id: 시리즈 ID")
        print(f"   ✅ series_name: 시리즈 이름")
        print(f"   ✅ season_number: 시즌 번호")
        print(f"   ✅ name: 시즌 이름")
        print(f"   ✅ air_date: 첫 방영일")
        print(f"   ✅ overview: 시즌 개요")
        print(f"   ✅ vote_average: 평균 평점")
        print(f"   ✅ vote_count: 투표 수")
        print(f"   ✅ network_names: 방송사 이름")
        print(f"   ✅ network_ids: 방송사 ID")
        print(f"   ✅ total_episodes: 총 에피소드 수")
        print(f"   ✅ avg_episode_runtime: 평균 에피소드 런타임")
        print("="*90)
    else:
        print("\n⚠️ 저장할 데이터가 없습니다.")